### Домашнее задание
#### Урок 2. Парсинг HTML. BeautifulSoup

Вариант 1
Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы) с сайтов Superjob и HH. Приложение должно анализировать несколько страниц сайта (также вводим через input или аргументы). Получившийся список должен содержать в себе минимум:
Наименование вакансии.
Предлагаемую зарплату (отдельно минимальную и максимальную).
Ссылку на саму вакансию.
Сайт, откуда собрана вакансия. ### По желанию можно добавить ещё параметры вакансии (например, работодателя и расположение). Структура должна быть одинаковая для вакансий с обоих сайтов. Общий результат можно вывести с помощью dataFrame через pandas.

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import lxml
import re
import transliterate

In [2]:
def hascyr(s):
    lower = set('абвгдеёжзийклмнопрстуфхцчшщъыьэюя')
    return lower.intersection(s.lower())!=set(lower)

In [3]:
def has_cyrillic(text):
    return bool(re.search('[а-яА-Я]', text))

In [4]:
def parse_hh(url, headers):    
    response = requests.get(url, headers=headers).text
    soup = BeautifulSoup(response, 'lxml')    
    
    v_name_soup = 'body ' + 'div.vacancy-serp-item__info ' + 'span ' * 3 + 'a'    #'div ' * 16 + 
    v_link_soup = v_name_soup + ' href'
    v_sum_soup = 'body ' + 'div.vacancy-serp-item__sidebar ' + 'span' #'div ' * 15 +
    v_firm_soup = 'body ' +  'div.vacancy-serp-item__meta-info-company '+ 'a' #'div ' * 18 +
    v_adress_soup = 'body ' +  'div.vacancy-serp-item__meta-info ' + 'span.vacancy-serp-item__meta-info' #'div ' * 18 +
    
    v_name = [i.text for i in soup.select(v_name_soup)]    
    v_link = [i['href'] for i in soup.select(v_name_soup)]    
    
    v_firm = [i.text for i in soup.select(v_firm_soup)]    
    v_adress = [i.text for i in soup.select(v_adress_soup)]    
    
    data_to_return = []
    for num, i in enumerate(v_name):
        data_to_return.append({
            "name": v_name[num],
            "link": v_link[num],
            "sum_min": 0, #v_sum[num].replace('\u202f', ''),
            "sum_max": 0, #v_sum[num].replace('\u202f', ''),
            "firm": v_firm[num],
            "adress": v_adress[num],
            "site": 'hh.ru'#,
            #"text":''
        })  
    
    return data_to_return

In [5]:
def parse_hh_sum(url, headers):    
    response = requests.get(url, headers=headers).text
    soup = BeautifulSoup(response, 'lxml') 
    txt=[i.text for i in soup.select('div.vacancy-description')]
    find_min = response.find('vac_salary_from')
    findmax = response.find('vac_salary_to')
    findCur = response.find('vac_salary_cur')
    strmin =response[find_min : findmax]
    s_min=strmin[(strmin.index(":")+3):(strmin.index("\n")-2)]
    if len(s_min)==0:
        s_min=0
    else:
        s_min=int(s_min)
    strmax=response[findmax : findCur]
    s_max=strmax[(strmax.index(":")+3):(strmax.index("\n")-2)]
    if len(s_max)==0:
        s_max=0
    else:
        s_max=int(s_max)
    return s_min,s_max #, txt

In [6]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.90 Safari/537.36'
}

In [7]:
text_vacancy = input("Какую вакансию ищем:") #find_vacancy = 'data+scientist'
find_vacancy_hh = text_vacancy.replace(' ', '+')
find_vacancy_hh

Какую вакансию ищем:data


'data'

In [8]:
col_list = int(input("Сколько страниц смотрим 1- 5:"))
col_list

Сколько страниц смотрим 1- 5:2


2

## Find hh.ru

In [9]:
all_hh=[]
for i in range(1, col_list+1):    
    url = 'https://www.hh.ru/search/vacancy?clusters=true&area={}&enable_snippets=true&salary=&st=searchVacancy&text='.format(i) + find_vacancy_hh
    #ссылка с ценой
    #'https://hh.ru/search/vacancy?clusters=true&enable_snippets=true&st=searchVacancy&text=data++scientist&area=1&salary=205000&only_with_salary=true'
    #print(url)
   
    hh_temp = parse_hh(url, headers)    
    for j in hh_temp:
        url2=j['link']
        #print(url2)
        j['sum_min'], j['sum_max'] = parse_hh_sum(url2, headers)
        
    all_hh += hh_temp

In [10]:
pd.DataFrame(all_hh).to_csv('vacancy_hh.csv', index=False, encoding="utf-8")

all_hh

## Find superjob.ru

In [12]:
def parse_sj(url, headers):
    response = requests.get(url, headers=headers).text
    soup = BeautifulSoup(response, 'lxml')   
    
    data_to_return_sj =[]   
    #найдено вакансий
    for i in soup.findAll('span', {'class':"_1h3Zg _2xFS1 _2hCDz _2ZsgW"}):
        print(i.text)
    # ссылки на вакансии
    for j in soup.findAll('div', {'class':"_1h3Zg _2rfUm _2hCDz _21a7u"}):
        #print(j.a)    
        #print("www.superjob.ru" + j.a['href'])    
        data_to_return_sj.append({
            "name": "",
            "link": "https://www.superjob.ru" + j.a['href'],
            "sum_min": 0, 
            "sum_max": 0, 
            "firm": "",
            "adress": "",
            "site": 'superjob.ru'#,
            #"text":''
        }) 
    
            
    return data_to_return_sj

In [13]:
find_vacancy_sj = text_vacancy.replace(' ', '-')
if has_cyrillic(find_vacancy_sj)== True:
    find_vacancy_sj = transliterate.translit(find_vacancy_sj, reversed=True)
print(find_vacancy_sj)

all_sj=[]

for i in range(1, col_list+1):    
    if i == 1:
        url = f'https://www.superjob.ru/vacancy/search/?keywords={find_vacancy_sj}&geo%5Bt%5D%5B0%5D=4'
    else:
        url = f'https://www.superjob.ru/vacancy/search/?keywords={find_vacancy_sj}&geo%5Bt%5D%5B0%5D=4&page={i}'
    
    #https://www.superjob.ru/vacancy/search/?keywords=data&geo%5Bt%5D%5B0%5D=4
    #https://www.superjob.ru/vacancy/search/?keywords=data&geo%5Bt%5D%5B0%5D=4&page=2
    
    print(url)
   
    sj_temp = parse_sj(url, headers)        
    
    #заполнение данными
    for j in sj_temp:
        url2=j['link']        
        
        response = requests.get(url2, headers=headers).text
        soup = BeautifulSoup(response, 'lxml')       
        #название вакансии
        for d in soup.findAll('h1', {'class':"_1h3Zg rFbjy _2dazi _2hCDz"}):
            j['name'] = (d.text)            
                
        #местонахождение
        for d in soup.findAll('span', {"_6-z9f"}):
            j['adress'] = (d.span.text)        
        
        #оклад
        salary = ""
        for d1 in soup.findAll('span', {'class':"_1h3Zg _2Wp8I _2rfUm _2hCDz"}):
            salary += d1.text
            
        salary=salary.replace(u'\xa0', u'')
        
        if '—' in salary:
                    salary_min = salary.split('—')[0]
                    salary_min = re.sub(r'[^0-9]', '', salary_min)
                    salary_max = salary.split('—')[1]
                    salary_max = re.sub(r'[^0-9]', '', salary_max)
                    salary_min = int(salary_min)
                    salary_max = int(salary_max)
        elif 'от' in salary:
                    salary_min = salary[2:]
                    salary_min = re.sub(r'[^0-9]', '', salary_min)
                    salary_min = int(salary_min)
                    salary_max = None
        elif 'договорённости' in salary:
                    salary_min = None
                    salary_max = None
        elif 'до' in salary:
                    salary_min = None
                    salary_max = salary[2:]
                    salary_max = re.sub(r'[^0-9]', '', salary_max)
                    salary_max = int(salary_max)
        else:
                    salary_min = int(re.sub(r'[^0-9]', '', salary))
                    salary_max = int(re.sub(r'[^0-9]', '', salary))    
            
            
        j['sum_min'] = salary_min
        j['sum_max'] = salary_max
        
        #содержание вакансии
        #txt=""
        #for d2 in soup.findAll('span', {'class':"_1h3Zg _2LeqZ _1TK9I _2hCDz _2ZsgW _2SvHc"}):
        #    txt+=d2.text         
        #j['text'] = txt
        
        #компания
        for d in soup.findAll('h2', {'class':"_1h3Zg _2rfUm _2hCDz _2ZsgW _21a7u _2SvHc"}):
            j['firm'] = (d.text)
            
    all_sj += sj_temp

data
https://www.superjob.ru/vacancy/search/?keywords=data&geo%5Bt%5D%5B0%5D=4
Найдено 59 вакансий
https://www.superjob.ru/vacancy/search/?keywords=data&geo%5Bt%5D%5B0%5D=4&page=2
Найдено 59 вакансий


all_sj


Вариант 2
Необходимо собрать информацию по продуктам питания с сайтов:
Роскачество официальный сайт. Исследование качества продуктов питания | Рейтинг товаров.
Список протестированных продуктов на сайте Росконтроль.рф
Получившийся список должен содержать:
Наименование продукта.
Категорию продукта (например «Бакалея»).
Подкатегорию продукта (например «Рис круглозерный»).
Параметр «Безопасность».
Параметр «Качество».
Общий балл.
Сайт, откуда получена информация. ### Структура должна быть одинаковая для продуктов с обоих сайтов. Общий результат можно вывести с помощью dataFrame через Pandas.

In [15]:
pd.DataFrame(all_sj).to_csv('vacancy_sj.csv', index=False, encoding="utf-8")